In [ ]:
# The code was removed by Watson Studio for sharing.

# **IBM APPLIED DATA SCIENCE CAPSTONE PROJECT**

## Explore and Cluster the Neighborhoods in Toronto 

### A. Web Scraping and Dataframe Creation

#### 1. Use Beautifulsoup to Scrap the Web and Create the Required Dataframe

In [20]:
#import pandas, numpy, Beatifulsoup
import pandas as pd
import numpy as np
import requests

In [21]:
#install beautifulsoup
! conda install -c conda-forge beautifulsoup4 --yes
print("beautifulSoup library intalled")

Solving environment: done

# All requested packages already installed.

beautifulSoup library intalled


In [22]:
#beautifusoup
from bs4 import BeautifulSoup

#create an object with the data from the Wikipedia page
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url)

In [23]:
#find the table in the website
myTable = soup.find('table',{'class':'wikitable sortable'})

In [24]:
#extract the table data of Wikipedia
wiki_data = [] 
for row in myTable.find_all("tr"):
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    wiki_data.append(cols)

#### 2. Create a Dataframe with the Borough Name and Neighborhood Names in Toronto

In [25]:
#create a panda dataframe
import pandas as pd
df = pd.DataFrame(wiki_data)

#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = df.rename(columns={0:"PostalCode",1:"Borough",2:"Neighborhood"})
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [26]:
#Only process the cells that have an assigned borough:
#1)drop the first row
df2 = df.copy()
df2 = df.drop([0])
#2)drop cells with a borough that is Not assigned
df2 = df2.drop(df2[df2['Borough']=='Not assigned'].index)

df2 = df2.reset_index(drop=True)
df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [27]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df3 = df2.copy()
df3['Neighborhood'] = df3.apply(
    lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'],
    axis=1
)

In [30]:
#if there is more than one neighborhood in one postal code area, combine those rows and separate the neighborhoods with a comma (group the dataframe by postcode and borough,and then join the neighborhoods)
df4 = df3.copy()
df4 = (df4.groupby(['PostalCode','Borough'])['Neighborhood']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())

df4 = pd.DataFrame(df4)

df4.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [32]:
df4.shape

(103, 3)

In [ ]:
# The code was removed by Watson Studio for sharing.